Question 1: [Index] S&P 500 Stocks Added to the Index
Which year had the highest number of additions?

Using the list of S&P 500 companies from Wikipedia's S&P 500 companies page, download the data including the year each company was added to the index.

Hint: you can use pandas.read_html to scrape the data into a DataFrame.

Steps:

Create a DataFrame with company tickers, names, and the year they were added.
Extract the year from the addition date and calculate the number of stocks added each year.
Which year had the highest number of additions (1957 doesn't count, as it was the year when the S&P 500 index was founded)? Write down this year as your answer (the most recent one, if you have several records).
Context:

"Following the announcement, all four new entrants saw their stock prices rise in extended trading on Friday" - recent examples of S&P 500 additions include DASH, WSM, EXE, TKO in 2025 (Nasdaq article).

Additional: How many current S&P 500 stocks have been in the index for more than 20 years? When stocks are added to the S&P 500, they usually experience a price bump as investors and index funds buy shares following the announcement.

In [1]:
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df = pd.read_html(url)[0]

df1 = df[["Symbol","Security"]].copy() 
df1["YearAdded"] = pd.to_datetime(df["Date added"]).dt.year
print(df1["YearAdded"].value_counts().sort_values(ascending=False).head(4))

YearAdded
1957    53
2017    23
2016    23
2019    22
Name: count, dtype: int64


In [3]:
year_diff = (pd.to_datetime('today') - pd.to_datetime(df["Date added"])) / pd.Timedelta(days=365.25)
print(len(year_diff[year_diff >= 20]), " in last 20 years")

219  in last 20 years


Question 2. [Macro] Indexes YTD (as of 1 May 2025)
How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?

Using Yahoo Finance World Indices data, compare the year-to-date (YTD) performance (1 January-1 May 2025) of major stock market indexes for the following countries:

United States - S&P 500 (^GSPC)
China - Shanghai Composite (000001.SS)
Hong Kong - HANG SENG INDEX (^HSI)
Australia - S&P/ASX 200 (^AXJO)
India - Nifty 50 (^NSEI)
Canada - S&P/TSX Composite (^GSPTSE)
Germany - DAX (^GDAXI)
United Kingdom - FTSE 100 (^FTSE)
Japan - Nikkei 225 (^N225)
Mexico - IPC Mexico (^MXX)
Brazil - Ibovespa (^BVSP)
Hint: use start_date='2025-01-01' and end_date='2025-05-01' when downloading daily data in yfinance

Context:

Global Valuations: Who's Cheap, Who's Not? article suggests "Other regions may be growing faster than the US and you need to diversify."

Reference: Yahoo Finance World Indices - https://finance.yahoo.com/world-indices/

Additional: How many of these indexes have better returns than the S&P 500 over 3, 5, and 10 year periods? Do you see the same trend? Note: For simplicity, ignore currency conversion effects.)

In [4]:
import yfinance as yf
import pandas as pd

In [5]:
## These stocks close on two different dates for 2024, joining them into one
df_yearclose1 = yf.download(tickers = ["^GSPC", "^HSI", "000001.SS", "^AXJO", "^NSEI", "^GSPTSE", "^FTSE", "^MXX"], start = '2024-12-31', end = '2025-01-01', interval = "1d")
df_yearclose2 = yf.download(tickers = ['^BVSP', '^N225', '^GDAXI'], start = '2024-12-30', end = '2025-01-02', interval = "1d")
df_yearclose = pd.concat([df_yearclose1, df_yearclose2], ignore_index=True)

## Rearranging into a better format
yearclose_df = df_yearclose.loc[:, ('Close', slice(None))]
yearclose_df.columns = yearclose_df.columns.droplevel()
yearclose_df = yearclose_df.stack().reset_index()
yearclose_df.columns = ['Date', 'Ticker', 'Close2024']
yearclose_df.drop('Date', axis=1, inplace=True)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  8 of 8 completed
[*********************100%***********************]  3 of 3 completed


In [6]:
## These stocks close on two different dates for closest to 5/1/2025, joining them into one
df_may1 = yf.download(tickers = ["^GSPC", "^HSI", "000001.SS", "^AXJO", "^NSEI", "^GSPTSE", "^FTSE", "^MXX", '^BVSP', '^N225', '^GDAXI'], start = '2025-05-01', end = '2025-05-02', interval = "1d")
df_may2 = yf.download(tickers = ['^BVSP', '000001.SS', '^MXX', '^NSEI', '^HSI', '^GDAXI'], start = '2025-04-30', end = '2025-05-01', interval = "1d")
df_may = pd.concat([df_may1, df_may2], ignore_index=True)

## Rearranging into a better format
may_df = df_may.loc[:, ('Close', slice(None))]
may_df.columns = may_df.columns.droplevel()
may_df = may_df.stack().reset_index()
may_df.columns = ['Date', 'Ticker', 'May2025']
may_df.drop('Date', axis=1, inplace=True)

[*********************100%***********************]  11 of 11 completed

6 Failed downloads:
['^HSI', '^NSEI', '^BVSP', '^GDAXI', '^MXX', '000001.SS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-05-01 -> 2025-05-02)')
[*********************100%***********************]  6 of 6 completed


In [7]:
## Join the two to get initial and final prices and calculate change
merged_df = pd.merge(yearclose_df, may_df, on='Ticker')
merged_df['percent_change'] = (merged_df['May2025'] - merged_df['Close2024']) / merged_df['Close2024']
SP = merged_df[merged_df['Ticker'] == '^GSPC']['percent_change'].values[0]

print(merged_df.sort_values(by='percent_change'))
print("")
print("Count > GSPC:", len(merged_df[merged_df['percent_change'] > SP]))


       Ticker      Close2024        May2025  percent_change
10      ^N225   39894.539062   36452.300781       -0.086283
3       ^GSPC    5881.629883    5604.140137       -0.047179
0   000001.SS    3351.762939    3279.031006       -0.021700
1       ^AXJO    8159.100098    8145.600098       -0.001655
4     ^GSPTSE   24727.900391   24795.599609        0.002738
7       ^NSEI   23644.800781   24334.199219        0.029156
2       ^FTSE    8173.000000    8496.799805        0.039618
5        ^HSI   20059.949219   22119.410156        0.102665
8       ^BVSP  120283.000000  135067.000000        0.122910
9      ^GDAXI   19909.140625   22496.980469        0.129982
6        ^MXX   49513.269531   56259.281250        0.136247

Count > GSPC: 9


Question 3. [Index] S&P 500 Market Corrections Analysis
Calculate the median duration (in days) of significant market corrections in the S&P 500 index.

For this task, define a correction as an event when a stock index goes down by more than 5% from the closest all-time high maximum.

Steps:

Download S&P 500 historical data (1950-present) using yfinance
Identify all-time high points (where price exceeds all previous prices)
For each pair of consecutive all-time highs, find the minimum price in between
Calculate drawdown percentages: (high - low) / high × 100
Filter for corrections with at least 5% drawdown
Calculate the duration in days for each correction period
Determine the 25th, 50th (median), and 75th percentiles for correction durations
Context:

Investors often wonder about the typical length of market corrections when deciding "when to buy the dip" (Reddit discussion).
A Wealth of Common Sense - How Often Should You Expect a Stock Market Correction?
Hint (use this data to compare with your results): Here is the list of top 10 largest corrections by drawdown:

2007-10-09 to 2009-03-09: 56.8% drawdown over 517 days
2000-03-24 to 2002-10-09: 49.1% drawdown over 929 days
1973-01-11 to 1974-10-03: 48.2% drawdown over 630 days
1968-11-29 to 1970-05-26: 36.1% drawdown over 543 days
2020-02-19 to 2020-03-23: 33.9% drawdown over 33 days
1987-08-25 to 1987-12-04: 33.5% drawdown over 101 days
1961-12-12 to 1962-06-26: 28.0% drawdown over 196 days
1980-11-28 to 1982-08-12: 27.1% drawdown over 622 days
2022-01-03 to 2022-10-12: 25.4% drawdown over 282 days
1966-02-09 to 1966-10-07: 22.2% drawdown over 240 days

In [8]:
import yfinance as yf
import pandas as pd
import numpy as np


In [ ]:
'''
## This is a more appropriate way of doing it, however it looks like the homework uses close price, not actual high and lows

df_sp = yf.download(tickers = ["^GSPC"], start = '1950-01-01', end = '2025-06-01', interval = "1d")
df_sp = df_sp[['High', 'Low']]

df_sp.columns = [f'{level}_{col}' for level, col in df_sp.columns]
df_sp = df_sp.sort_values(by='Date').reset_index()
df_sp.columns = ['Date', 'High', 'Low']

df_sp['cummax'] = df_sp['High'].cummax()
df_sp['is_cummax'] = df_sp['High'] == df_sp['cummax']

df_sp['group'] = df_sp['is_cummax'].cumsum()
df_sp['group_min'] = df_sp.groupby('group')['Low'].transform('min')

'''

In [9]:
df_sp = yf.download(tickers = ["^GSPC"], start = '1950-01-01', end = '2025-06-01', interval = "1d")
df_sp = df_sp['Close']

df_sp = df_sp.sort_values(by='Date').reset_index()
df_sp.columns = ['Date', 'Close']

df_sp['cummax'] = df_sp['Close'].cummax()
df_sp['is_cummax'] = df_sp['Close'] == df_sp['cummax']

df_sp['group'] = df_sp['is_cummax'].cumsum()
df_sp['group_min'] = df_sp.groupby('group')['Close'].transform('min')
df_sp['group_min_day'] =  np.where(df_sp['group_min'] == df_sp['Close'], df_sp['Date'], None)
df_sp['group_min_day'] = pd.to_datetime(df_sp['group_min_day'])
df_sp['group_min_day'] = df_sp.groupby('group')['group_min_day'].transform('min')

df_sp_filtered = df_sp[df_sp['is_cummax']]

df_sp_filtered = df_sp_filtered[['Date', 'Close','group_min_day','group_min']]          
df_sp_filtered['percent_drop'] = (df_sp_filtered['Close'] - df_sp_filtered['group_min']) / df_sp_filtered['Close']
df_sp_filtered['day_diff'] = (df_sp_filtered['group_min_day'] - df_sp_filtered['Date']).dt.days
df_sp_filtered = df_sp_filtered[df_sp_filtered['percent_drop'] > .05]

print(df_sp_filtered['day_diff'].median())
df_sp_filtered.sort_values(by='percent_drop', ascending=False)

# Maybe a rounding difference somewhere? I get an even number of observation (72) with the median beiing the average of 39 and 42 = 40.5.
# The only answer close is 39


[*********************100%***********************]  1 of 1 completed

40.5


,Date,Close,group_min_day,group_min,percent_drop,day_diff
14534,2007-10-09,1565.150024,2009-03-09,676.530029,0.567754,517
12639,2000-03-24,1527.459961,2002-10-09,776.760010,0.491469,929
5765,1973-01-11,120.239998,1974-10-03,62.279999,0.482036,630
4731,1968-11-29,108.370003,1970-05-26,69.290001,0.360616,543
17645,2020-02-19,3386.149902,2020-03-23,2237.399902,0.339250,33
...,...,...,...,...,...,...
7709,1980-09-22,130.399994,1980-09-29,123.540001,0.052607,7
18034,2021-09-02,4536.950195,2021-10-04,4300.459961,0.052125,32
5663,1972-08-14,112.550003,1972-10-16,106.769997,0.051355,63
12397,1999-04-12,1358.630005,1999-04-19,1289.479980,0.050897,7


Question 4. [Stocks] Earnings Surprise Analysis for Amazon (AMZN)
Calculate the median 2-day percentage change in stock prices following positive earnings surprises days.

Steps:

Load earnings data from CSV (ha1_Amazon.csv) containing earnings dates, EPS estimates, and actual EPS. Make sure you are using the correct delimiter to read the data, such as in this command python pandas.read_csv("ha1_Amazon.csv", delimiter=';') 
Download complete historical price data using yfinance
Calculate 2-day percentage changes for all historical dates: for each sequence of 3 consecutive trading days (Day 1, Day 2, Day 3), compute the return as Close_Day3 / Close_Day1 - 1. (Assume Day 2 may correspond to the earnings announcement.)
Identify positive earnings surprises (where "actual EPS > estimated EPS" OR "Surprise (%)>0")
Calculate 2-day percentage changes following positive earnings surprises. Show your answer in % (closest number to the 2nd digit): return * 100.0
(Optional) Compare the median 2-day percentage change for positive surprises vs. all historical dates. Do you see the difference?
Context: Earnings announcements, especially when they exceed analyst expectations, can significantly impact stock prices in the short term.

Reference: Yahoo Finance earnings calendar - https://finance.yahoo.com/calendar/earnings?symbol=AMZN

Additional: Is there a correlation between the magnitude of the earnings surprise and the stock price reaction? Does the market react differently to earnings surprises during bull vs. bear markets?)

In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

In [ ]:
df_earnings = pd.read_csv("ha1_Amazon.csv", delimiter=';') 
df_earnings['EPS Estimate (Cleaned)'] = pd.to_numeric(df_earnings['EPS Estimate'].astype(str).str.replace(r'^(?!-)(\D+)|[^\d\.-]+', '', regex=True), errors='coerce')
df_earnings['Reported EPS (Cleaned)'] = pd.to_numeric(df_earnings['Reported EPS'].astype(str).str.replace(r'^(?!-)(\D+)|[^\d\.-]+', '', regex=True), errors='coerce')
df_earnings['Surprise (%) (Cleaned)'] = pd.to_numeric(df_earnings['Surprise (%)'], errors='coerce')
df_earnings['Date'] = pd.to_datetime(df_earnings['Earnings Date'].str.replace('EDT', 'EST', regex=False)).dt.normalize()


df_prices = yf.download(tickers = ["AMZN"], interval = "1d")['Close']
df_prices = df_prices.sort_values(by='Date').reset_index()
df_prices.columns = ['Date', 'Close']
df_prices['two_day_percent'] = (df_prices['Close'].shift(-1) / df_prices['Close'].shift(1)) - 1


df_earnings['BeatEarning'] = (df_earnings['Reported EPS (Cleaned)'] > df_earnings['EPS Estimate (Cleaned)'])
merged_df = pd.merge(df_earnings, df_prices, on='Date')

print(len(merged_df[merged_df['BeatEarning']]['two_day_percent']))
print(merged_df[merged_df['BeatEarning']]['two_day_percent'].median())
#get .010405 -- which is apparently wrong?
#merged_df[merged_df['BeatEarning']]


C:\Users\chris\AppData\Local\Temp\ipykernel_20664\2647395350.py:5: FutureWarning: Parsed string "April 29, 2026 at 6 AM EST" included an un-recognized timezone "EST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df_earnings['Date'] = pd.to_datetime(df_earnings['Earnings Date'].str.replace('EDT', 'EST', regex=False)).dt.normalize()
[*********************100%***********************]  1 of 1 completed

37
0.025702644458225343


,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%),EPS Estimate (Cleaned),Reported EPS (Cleaned),Surprise (%) (Cleaned),Date,BeatEarning,Close,two_day_percent
0,AMZN,"Amazon.com, Inc.","May 1, 2025 at 4 PM EDT",???.36,???.59,+16.74,36.00,59.00,16.74,2025-05-01,True,190.199997,0.030149
1,AMZN,"Amazon.com, Inc.","February 6, 2025 at 4 PM EST",???.49,???.86,+24.47,49.00,86.00,24.47,2025-02-06,True,238.830002,-0.029724
2,AMZN,"Amazon.com, Inc.","October 31, 2024 at 4 PM EDT",???.14,???.43,+25.17,14.00,43.00,25.17,2024-10-31,True,186.399994,0.026981
3,AMZN,"Amazon.com, Inc.","August 1, 2024 at 4 PM EDT",01.???,???.26,+22.58,1.00,26.00,22.58,2024-08-01,True,184.070007,-0.102043
4,AMZN,"Amazon.com, Inc.","April 30, 2024 at 4 PM EDT",0.83,0.98,+17.91,0.83,0.98,17.91,2024-04-30,True,175.000000,-0.010831
5,AMZN,"Amazon.com, Inc.","February 1, 2024 at 4 PM EST",0.8,1,+24.55,0.80,1.00,24.55,2024-02-01,True,159.279999,0.107023
6,AMZN,"Amazon.com, Inc.","October 26, 2023 at 4 PM EDT",0.58,0.94,+60.85,0.58,0.94,60.85,2023-10-26,True,119.570000,0.052311
7,AMZN,"Amazon.com, Inc.","August 3, 2023 at 4 PM EDT",0.35,0.65,+85.73,0.35,0.65,85.73,2023-08-03,True,128.910004,0.088605
8,AMZN,"Amazon.com, Inc.","April 27, 2023 at 4 PM EDT",0.21,0.31,+46.36,0.21,0.31,46.36,2023-04-27,True,109.820000,0.004477
9,AMZN,"Amazon.com, Inc.","February 2, 2023 at 4 PM EST",0.18,0.25,+42.56,0.18,0.25,42.56,2023-02-02,True,112.910004,-0.016738
